In [1]:
%run network.ipynb

Looking in links: https://download.pytorch.org/whl/torch_stable.htmlNote: you may need to restart the kernel to use updated packages.



In [2]:
%run data_loader.ipynb

In [3]:
%run utils.ipynb

In [4]:
import torch.nn as nn

class CrowdCounter(nn.Module):
    def __init__(self):
        super(CrowdCounter, self).__init__()        
        self.DME = MCNN()        
        self.loss_fn = nn.MSELoss()
        
    @property
    def loss(self):
        return self.loss_mse
    
    def forward(self,  im_data, gt_data=None):        
        im_data = np_to_variable(im_data, is_cuda=True, is_training=self.training)   
        density_map = self.DME(im_data)
        
        if self.training:                        
            gt_data = np_to_variable(gt_data, is_cuda=True, is_training=self.training)            
            self.loss_mse = self.build_loss(density_map, gt_data)
            
        return density_map
    
    def build_loss(self, density_map, gt_data):
        loss = self.loss_fn(density_map, gt_data)
        return loss

In [5]:
def evaluate_model(trained_model, data_loader):
    net = CrowdCounter()
    load_net(trained_model, net)
    net.cuda()
    net.eval()
    mae = 0.0
    mse = 0.0
    for blob in data_loader:                        
        im_data = blob['data'].numpy()
        im_data = np.expand_dims(im_data, 1) 
        gt_data = blob['gt_density'].numpy()
        gt_data = np.expand_dims(gt_data, 1)
        density_map = net(im_data, gt_data)
        density_map = density_map.data.cpu().numpy()
        gt_count = np.sum(gt_data)
        et_count = np.sum(density_map)
        mae += abs(gt_count-et_count)
        mse += ((gt_count-et_count)*(gt_count-et_count))        
    mae = mae/len(data_loader)
    mse = np.sqrt(mse/len(data_loader))
    return mae,mse

In [6]:
output_dir = '../saved_models/'
train_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/shanghaitech_part_B_patches_9/train/'
train_gt_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/shanghaitech_part_B_patches_9/train_den/'
val_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/shanghaitech_part_B_patches_9/val/'
val_gt_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/shanghaitech_part_B_patches_9/val_den/'

image_paths = os.listdir(train_path)

for i in range(len(image_paths)):
    image_paths[i] = train_path + image_paths[i]

target_paths = os.listdir(train_gt_path)

for i in range(len(target_paths)):
    target_paths[i] = train_gt_path + target_paths[i]
    
image_paths_val = os.listdir(val_path)

for i in range(len(image_paths_val)):
    image_paths_val[i] = val_path + image_paths_val[i]

target_paths_val = os.listdir(val_gt_path)

for i in range(len(target_paths_val)):
    target_paths_val[i] = val_gt_path + target_paths_val[i]



In [7]:
#data_loader = ImageDataLoader(train_path, train_gt_path, shuffle=True, gt_downsample=True, pre_load=True)
#data_loader_val = ImageDataLoader(val_path, val_gt_path, shuffle=False, gt_downsample=True, pre_load=True)

transform = transforms.Compose(
    [
    #transforms.Resize((256, 172), interpolation=2),
    transforms.ToTensor(),
     #transforms.CenterCrop(10),
    #transforms.Normalize((.5, .5), (.5, .5))
    ]
)

trainset = MyDataset(image_paths, target_paths)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=70, shuffle=True, num_workers=0)

trainset_val = MyDataset(image_paths_val, target_paths_val)
trainloader_val = torch.utils.data.DataLoader(trainset_val, batch_size=50, shuffle=False, num_workers=0)

data_loader = iter(trainloader)
data_loader_val = iter(trainloader_val)


In [8]:
start_step = 0
end_step = 500
lr = 0.00001
momentum = 0.9
disp_interval = 500
log_interval = 250

In [9]:
rand_seed = 64678  
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)


# load net
net = CrowdCounter()
weights_normal_init(net, dev=0.01)
net.cuda()
net.train()

params = list(net.parameters())
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
import sys
from tqdm import tqdm

# training
train_loss = 0
step_cnt = 0
best_mae = sys.maxsize * 2 + 1
method = 'MCNN'
dataset_name = 'shtechA'

for epoch in range(start_step, end_step+1):    
    step = -1
    train_loss = 0
    run = tqdm(enumerate(data_loader, 0))
    for i, blob in run:                    
        step += 1 
        im_data = blob['data'].numpy()
        im_data = np.expand_dims(im_data, 1) 
        gt_data = blob['gt_density'].numpy()
        gt_data = np.expand_dims(gt_data, 1) 
        density_map = net(im_data, gt_data)
        loss = net.loss
        train_loss += loss.data
        step_cnt += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % disp_interval == 0:            
            gt_count = np.sum(gt_data)    
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            print('Crowd count(et, gt):', et_count, gt_count)
            #save_results(im_data,gt_data,density_map, output_dir)
            
        run.set_description("Train loss: {}".format(train_loss))
        run.refresh() # to show immediately the update
    
    data_loader = iter(trainloader)
    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method,dataset_name,epoch))
        save_net(save_name, net)     
        #calculate error on the validation dataset 
        mae,mse = evaluate_model(save_name, data_loader_val)
        data_loader_val = iter(trainloader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method,dataset_name,epoch)
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch,mae,mse)
        print(log_text)
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae,best_mse, best_model)
        print(log_text)

Train loss: 9.100467650569044e-07: : 1it [00:01,  1.78s/it]

Crowd count(et, gt): 933.56525 393.89545


Train loss: 3.6706293030874804e-05: : 47it [01:15,  1.61s/it]
0it [00:00, ?it/s]

EPOCH: 0, MAE: 163.5, MSE: 201.0
BEST MAE: 163.5, BEST MSE: 201.0, BEST MODEL: MCNN_shtechA_0.h5


Train loss: 8.986453963188978e-07: : 1it [00:01,  1.23s/it]

...

Train loss: 8.636231996206334e-07: : 1it [00:01,  1.33s/it]

Crowd count(et, gt): 301.8777 481.43927


Train loss: 3.393965744180605e-05: : 47it [00:57,  1.22s/it] 
Train loss: 8.061415428528562e-07: : 1it [00:01,  1.34s/it]

Crowd count(et, gt): 188.68968 382.01907


Train loss: 1.9997485651401803e-05: : 27it [00:34,  1.29s/it]